<a href="https://colab.research.google.com/github/JimKing100/NFL-Live/blob/master/data-science/actuals/Actuals_Offense.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
# Imports
import pandas as pd

In [0]:
# Load the raw data
player_df = pd.read_csv('https://raw.githubusercontent.com/JimKing100/NFL-Live/master/data-science/data/raw/players.csv')
offense_df = pd.read_csv('https://raw.githubusercontent.com/JimKing100/NFL-Live/master/data-science/data/raw/offense.csv')
td_df = pd.read_csv('https://raw.githubusercontent.com/JimKing100/NFL-Live/master/data-science/data/raw/td.csv')
game_df = pd.read_csv('https://raw.githubusercontent.com/JimKing100/NFL-Live/master/data-science/data/raw/game.csv')

In [3]:
# The players dataframe - includes all the offensive players for the 2019 NFL season
player_df.head()

,player,first,last,name,position1,start
0,SF,San Francisco,49ers,San Francisco 49ers,DF,2000
1,CHI,Chicago,Bears,Chicago Bears,DF,2000
2,CIN,Cincinnati,Bengals,Cincinnati Bengals,DF,2000
3,BUF,Buffalo,Bills,Buffalo Bills,DF,2000
4,DEN,Denver,Broncos,Denver Broncos,DF,2000


In [4]:
# The offense dataframe - raw offensive data for all 2019 QBs, RBs, WRs, TEs for seasons 2000-2019
offense_df.head()

,uid,gid,week,player,del,first,last,name,position1,pa,pc,py,ints,tdp,ra,sra,ry,tdr,trg,rec,recy,tdrec,ret,rety,tdret,fuml,peny,conv,snp,fp,fp2,fp3,game,seas,year,team
0,3599,175,13,TB-2300,KEEP,Tom,Brady,Tom Brady,QB,3,1,6,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.24,0.24,0.24,0,1,2000,NE
1,5841,284,2,TB-2300,KEEP,Tom,Brady,Tom Brady,QB,10,5,46,0,0,1,1,9,0,0,0,0,0,0,0,0,0,0,0,0,2.74,2.74,2.74,0,2,2001,NE
2,6050,295,3,TB-2300,KEEP,Tom,Brady,Tom Brady,QB,23,13,168,0,0,1,0,2,0,0,0,0,0,0,0,0,0,0,0,0,6.92,6.92,6.92,0,2,2001,NE
3,6330,308,4,TB-2300,KEEP,Tom,Brady,Tom Brady,QB,24,12,86,0,0,2,1,9,0,0,0,0,0,0,0,0,2,0,0,0,0.34,0.34,2.34,0,2,2001,NE
4,6672,325,5,TB-2300,KEEP,Tom,Brady,Tom Brady,QB,54,33,364,0,2,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,22.56,22.56,25.56,0,2,2001,NE


In [5]:
# The touchdown dataframe - raw touchdown data for rushing, receiving and passing TD plays of 40 yards and greater.
td_df.head()

,pid,psr,first_qb,last_qb,name_qb,gid,year,week,qtr,min,sec,dwn,yds,pts,player,first,last,name,type
0,75,CC-1400,Chris,Chandler,Chris Chandler,1,2000,1,2,4,36,2,44,7,TM-0900,Terance,Mathis,Terance Mathis,REC
1,98,CC-1400,Chris,Chandler,Chris Chandler,1,2000,1,3,15,0,1,48,7,SJ-0400,Shawn,Jefferson,Shawn Jefferson,REC
2,1051,CM-2200,Cade,McNown,Cade McNown,7,2000,1,3,15,0,3,48,7,MR-1400,Marcus,Robinson,Marcus Robinson,REC
3,1509,KC-2000,Kerry,Collins,Kerry Collins,10,2000,1,2,10,11,1,78,7,TB-0400,Tiki,Barber,Tiki Barber,RUSH
4,1819,TB-0200,Tony,Banks,Tony Banks,12,2000,1,1,4,13,1,53,7,QI-0100,Qadry,Ismail,Qadry Ismail,REC


In [6]:
# The game dataframe - raw game data to calculate bye weeks
game_df.head()

,gid,seas,wk,day,v,h,stad,temp,humd,wspd,wdir,cond,surf,ou,sprv,ptsv,ptsh
0,1,2000,1,SUN,SF,ATL,Georgia Dome,79.0,NaN,NaN,NaN,Dome,AstroTurf,42.5,7.0,28,36
1,2,2000,1,SUN,JAC,CLE,Cleveland Browns Stadium,78.0,63.0,9.0,NE,Sunny,Grass,38.0,-10.0,27,7
2,3,2000,1,SUN,PHI,DAL,Texas Stadium,109.0,19.0,5.0,S,Sunny,AstroTurf,40.0,6.0,41,14
3,4,2000,1,SUN,NYJ,GB,Lambeau Field,77.0,66.0,5.0,E,Mostly Cloudy,Grass,36.0,2.5,20,16
4,5,2000,1,SUN,IND,KC,Arrowhead Stadium,90.0,50.0,8.0,E,Mostly Sunny,Grass,44.0,-3.0,27,14


In [0]:
# Pull the stats to calculate the player points
def pull_stats(df, df1, df2, col, p, y, g):

    # Set empty values to NaN if prior to start year, otherwise zero
    if (df[col].loc[(df['player']==p) & (df['year']==y) & (df['week']==g)].empty):
        start_year = df1['start'].loc[(df1['player']==p)].iloc[0]
        if y < start_year:
            points = float('NaN')
        else:
            points = 0
    else:
        rush_tds = df['tdr'].loc[(df['player']==p) & (df['year']==y) & (df['week']==g)].iloc[0]
        receive_tds = df['tdrec'].loc[(df['player']==p) & (df['year']==y) & (df['week']==g)].iloc[0]
        pass_tds = df['tdp'].loc[(df['player']==p) & (df['year']==y) & (df['week']==g)].iloc[0]
        return_tds = df['tdret'].loc[(df['player']==p) & (df['year']==y) & (df['week']==g)].iloc[0] 
        convs = df['conv'].loc[(df['player']==p) & (df['year']==y) & (df['week']==g)].iloc[0]
        rush_yds = df['ry'].loc[(df['player']==p) & (df['year']==y) & (df['week']==g)].iloc[0]
        receive_yds = df['recy'].loc[(df['player']==p) & (df['year']==y) & (df['week']==g)].iloc[0]
        pass_yds = df['py'].loc[(df['player']==p) & (df['year']==y) & (df['week']==g)].iloc[0]
        ints = df['ints'].loc[(df['player']==p) & (df['year']==y) & (df['week']==g)].iloc[0]
        fumbles = df['fuml'].loc[(df['player']==p) & (df['year']==y) & (df['week']==g)].iloc[0]
        rush_rec_td40 = df2['name'].loc[(df2['player']==p) & (df2['year']==y) & (df2['week']==g)].count()
        pass_td40 = df2['psr'].loc[(df2['psr']==p) & (df2['year']==y) & (df2['week']==g)].count()
        

        points = (rush_tds * 6) + \
                 (receive_tds * 6) + \
                 (pass_tds * 4) + \
                 (return_tds * 6) + \
                 (convs * 2) + \
                 (rush_yds / 10) + \
                 (receive_yds /10) + \
                 (pass_yds / 25) - \
                 (ints * 2) - \
                 (fumbles * 2) + \
                 (rush_rec_td40 * 2) + \
                 (pass_td40)

    return points

In [0]:
# Add a row to the final_df dataframe
# Each row represents the actual offensive points for each game for each offensive player
def add_row(df, p, f, l, n, pos, value_list):

    df = df.append({'player': p,
                    'first': f,
                    'last': l,
                    'name': n,
                    'position1': pos,
                    '2000-game1': value_list[0], 
                    '2000-game2': value_list[1],
                    '2000-game3': value_list[2], 
                    '2000-game4': value_list[3], 
                    '2000-game5': value_list[4], 
                    '2000-game6': value_list[5], 
                    '2000-game7': value_list[6], 
                    '2000-game8': value_list[7], 
                    '2000-game9': value_list[8],
                    '2000-game10': value_list[9],
                    '2000-game11': value_list[10],
                    '2000-game12': value_list[11],
                    '2000-game13': value_list[12],
                    '2000-game14': value_list[13],
                    '2000-game15': value_list[14],
                    '2000-game16': value_list[15],
                    '2001-game1': value_list[16], 
                    '2001-game2': value_list[17],
                    '2001-game3': value_list[18], 
                    '2001-game4': value_list[19], 
                    '2001-game5': value_list[20], 
                    '2001-game6': value_list[21], 
                    '2001-game7': value_list[22], 
                    '2001-game8': value_list[23], 
                    '2001-game9': value_list[24],
                    '2001-game10': value_list[25],
                    '2001-game11': value_list[26],
                    '2001-game12': value_list[27],
                    '2001-game13': value_list[28],
                    '2001-game14': value_list[29],
                    '2001-game15': value_list[30],
                    '2001-game16': value_list[31],
                    '2002-game1': value_list[32], 
                    '2002-game2': value_list[33],
                    '2002-game3': value_list[34], 
                    '2002-game4': value_list[35], 
                    '2002-game5': value_list[36], 
                    '2002-game6': value_list[37], 
                    '2002-game7': value_list[38], 
                    '2002-game8': value_list[39], 
                    '2002-game9': value_list[40],
                    '2002-game10': value_list[41],
                    '2002-game11': value_list[42],
                    '2002-game12': value_list[43],
                    '2002-game13': value_list[44],
                    '2002-game14': value_list[45],
                    '2002-game15': value_list[46],
                    '2002-game16': value_list[47],
                    '2003-game1': value_list[48], 
                    '2003-game2': value_list[49],
                    '2003-game3': value_list[50], 
                    '2003-game4': value_list[51], 
                    '2003-game5': value_list[52], 
                    '2003-game6': value_list[53], 
                    '2003-game7': value_list[54], 
                    '2003-game8': value_list[55], 
                    '2003-game9': value_list[56],
                    '2003-game10': value_list[57],
                    '2003-game11': value_list[58],
                    '2003-game12': value_list[59],
                    '2003-game13': value_list[60],
                    '2003-game14': value_list[61],
                    '2003-game15': value_list[62],
                    '2003-game16': value_list[63],
                    '2004-game1': value_list[64], 
                    '2004-game2': value_list[65],
                    '2004-game3': value_list[66], 
                    '2004-game4': value_list[67], 
                    '2004-game5': value_list[68], 
                    '2004-game6': value_list[69], 
                    '2004-game7': value_list[70], 
                    '2004-game8': value_list[71], 
                    '2004-game9': value_list[72],
                    '2004-game10': value_list[73],
                    '2004-game11': value_list[74],
                    '2004-game12': value_list[75],
                    '2004-game13': value_list[76],
                    '2004-game14': value_list[77],
                    '2004-game15': value_list[78],
                    '2004-game16': value_list[79],
                    '2005-game1': value_list[80], 
                    '2005-game2': value_list[81],
                    '2005-game3': value_list[82], 
                    '2005-game4': value_list[83], 
                    '2005-game5': value_list[84], 
                    '2005-game6': value_list[85], 
                    '2005-game7': value_list[86], 
                    '2005-game8': value_list[87], 
                    '2005-game9': value_list[88],
                    '2005-game10': value_list[89],
                    '2005-game11': value_list[90],
                    '2005-game12': value_list[91],
                    '2005-game13': value_list[92],
                    '2005-game14': value_list[93],
                    '2005-game15': value_list[94],
                    '2005-game16': value_list[95],
                    '2006-game1': value_list[96], 
                    '2006-game2': value_list[97],
                    '2006-game3': value_list[98], 
                    '2006-game4': value_list[99], 
                    '2006-game5': value_list[100], 
                    '2006-game6': value_list[101], 
                    '2006-game7': value_list[102], 
                    '2006-game8': value_list[103], 
                    '2006-game9': value_list[104],
                    '2006-game10': value_list[105],
                    '2006-game11': value_list[106],
                    '2006-game12': value_list[107],
                    '2006-game13': value_list[108],
                    '2006-game14': value_list[109],
                    '2006-game15': value_list[110],
                    '2006-game16': value_list[111],
                    '2007-game1': value_list[112], 
                    '2007-game2': value_list[113],
                    '2007-game3': value_list[114], 
                    '2007-game4': value_list[115], 
                    '2007-game5': value_list[116], 
                    '2007-game6': value_list[117], 
                    '2007-game7': value_list[118], 
                    '2007-game8': value_list[119], 
                    '2007-game9': value_list[120],
                    '2007-game10': value_list[121],
                    '2007-game11': value_list[122],
                    '2007-game12': value_list[123],
                    '2007-game13': value_list[124],
                    '2007-game14': value_list[125],
                    '2007-game15': value_list[126],
                    '2007-game16': value_list[127],
                    '2008-game1': value_list[128], 
                    '2008-game2': value_list[129],
                    '2008-game3': value_list[130], 
                    '2008-game4': value_list[131], 
                    '2008-game5': value_list[132], 
                    '2008-game6': value_list[133], 
                    '2008-game7': value_list[134], 
                    '2008-game8': value_list[135], 
                    '2008-game9': value_list[136],
                    '2008-game10': value_list[137],
                    '2008-game11': value_list[138],
                    '2008-game12': value_list[139],
                    '2008-game13': value_list[140],
                    '2008-game14': value_list[141],
                    '2008-game15': value_list[142],
                    '2008-game16': value_list[143],
                    '2009-game1': value_list[144], 
                    '2009-game2': value_list[145],
                    '2009-game3': value_list[146], 
                    '2009-game4': value_list[147], 
                    '2009-game5': value_list[148], 
                    '2009-game6': value_list[149], 
                    '2009-game7': value_list[150], 
                    '2009-game8': value_list[151], 
                    '2009-game9': value_list[152],
                    '2009-game10': value_list[153],
                    '2009-game11': value_list[154],
                    '2009-game12': value_list[155],
                    '2009-game13': value_list[156],
                    '2009-game14': value_list[157],
                    '2009-game15': value_list[158],
                    '2009-game16': value_list[159],
                    '2010-game1': value_list[160], 
                    '2010-game2': value_list[161],
                    '2010-game3': value_list[162], 
                    '2010-game4': value_list[163], 
                    '2010-game5': value_list[164], 
                    '2010-game6': value_list[165], 
                    '2010-game7': value_list[166], 
                    '2010-game8': value_list[167], 
                    '2010-game9': value_list[168],
                    '2010-game10': value_list[169],
                    '2010-game11': value_list[170],
                    '2010-game12': value_list[171],
                    '2010-game13': value_list[172],
                    '2010-game14': value_list[173],
                    '2010-game15': value_list[174],
                    '2010-game16': value_list[175],
                    '2011-game1': value_list[176], 
                    '2011-game2': value_list[177],
                    '2011-game3': value_list[178], 
                    '2011-game4': value_list[179], 
                    '2011-game5': value_list[180], 
                    '2011-game6': value_list[181], 
                    '2011-game7': value_list[182], 
                    '2011-game8': value_list[183], 
                    '2011-game9': value_list[184],
                    '2011-game10': value_list[185],
                    '2011-game11': value_list[186],
                    '2011-game12': value_list[187],
                    '2011-game13': value_list[188],
                    '2011-game14': value_list[189],
                    '2011-game15': value_list[190],
                    '2011-game16': value_list[191],
                    '2012-game1': value_list[192], 
                    '2012-game2': value_list[193],
                    '2012-game3': value_list[194], 
                    '2012-game4': value_list[195], 
                    '2012-game5': value_list[196], 
                    '2012-game6': value_list[197], 
                    '2012-game7': value_list[198], 
                    '2012-game8': value_list[199], 
                    '2012-game9': value_list[200],
                    '2012-game10': value_list[201],
                    '2012-game11': value_list[202],
                    '2012-game12': value_list[203],
                    '2012-game13': value_list[204],
                    '2012-game14': value_list[205],
                    '2012-game15': value_list[206],
                    '2012-game16': value_list[207],
                    '2013-game1': value_list[208], 
                    '2013-game2': value_list[209],
                    '2013-game3': value_list[210], 
                    '2013-game4': value_list[211], 
                    '2013-game5': value_list[212], 
                    '2013-game6': value_list[213], 
                    '2013-game7': value_list[214], 
                    '2013-game8': value_list[215], 
                    '2013-game9': value_list[216],
                    '2013-game10': value_list[217],
                    '2013-game11': value_list[218],
                    '2013-game12': value_list[219],
                    '2013-game13': value_list[220],
                    '2013-game14': value_list[221],
                    '2013-game15': value_list[222],
                    '2013-game16': value_list[223],
                    '2014-game1': value_list[224], 
                    '2014-game2': value_list[225],
                    '2014-game3': value_list[226], 
                    '2014-game4': value_list[227], 
                    '2014-game5': value_list[228], 
                    '2014-game6': value_list[229], 
                    '2014-game7': value_list[230], 
                    '2014-game8': value_list[231], 
                    '2014-game9': value_list[232],
                    '2014-game10': value_list[233],
                    '2014-game11': value_list[234],
                    '2014-game12': value_list[235],
                    '2014-game13': value_list[236],
                    '2014-game14': value_list[237],
                    '2014-game15': value_list[238],
                    '2014-game16': value_list[239],
                    '2015-game1': value_list[240], 
                    '2015-game2': value_list[241],
                    '2015-game3': value_list[242], 
                    '2015-game4': value_list[243], 
                    '2015-game5': value_list[244], 
                    '2015-game6': value_list[245], 
                    '2015-game7': value_list[246], 
                    '2015-game8': value_list[247], 
                    '2015-game9': value_list[248],
                    '2015-game10': value_list[249],
                    '2015-game11': value_list[250],
                    '2015-game12': value_list[251],
                    '2015-game13': value_list[252],
                    '2015-game14': value_list[253],
                    '2015-game15': value_list[254],
                    '2015-game16': value_list[255],
                    '2016-game1': value_list[256], 
                    '2016-game2': value_list[257],
                    '2016-game3': value_list[258], 
                    '2016-game4': value_list[259], 
                    '2016-game5': value_list[260], 
                    '2016-game6': value_list[261], 
                    '2016-game7': value_list[262], 
                    '2016-game8': value_list[263], 
                    '2016-game9': value_list[264],
                    '2016-game10': value_list[265],
                    '2016-game11': value_list[266],
                    '2016-game12': value_list[267],
                    '2016-game13': value_list[268],
                    '2016-game14': value_list[269],
                    '2016-game15': value_list[270],
                    '2016-game16': value_list[271],
                    '2017-game1': value_list[272], 
                    '2017-game2': value_list[273],
                    '2017-game3': value_list[274], 
                    '2017-game4': value_list[275], 
                    '2017-game5': value_list[276], 
                    '2017-game6': value_list[277], 
                    '2017-game7': value_list[278], 
                    '2017-game8': value_list[279], 
                    '2017-game9': value_list[280],
                    '2017-game10': value_list[281],
                    '2017-game11': value_list[282],
                    '2017-game12': value_list[283],
                    '2017-game13': value_list[284],
                    '2017-game14': value_list[285],
                    '2017-game15': value_list[286],
                    '2017-game16': value_list[287],
                    '2018-game1': value_list[288], 
                    '2018-game2': value_list[289],
                    '2018-game3': value_list[290], 
                    '2018-game4': value_list[291], 
                    '2018-game5': value_list[292], 
                    '2018-game6': value_list[293], 
                    '2018-game7': value_list[294], 
                    '2018-game8': value_list[295], 
                    '2018-game9': value_list[296],
                    '2018-game10': value_list[297],
                    '2018-game11': value_list[298],
                    '2018-game12': value_list[299],
                    '2018-game13': value_list[300],
                    '2018-game14': value_list[301],
                    '2018-game15': value_list[302],
                    '2018-game16': value_list[303],
                    '2019-game1': value_list[304], 
                    '2019-game2': value_list[305],
                    '2019-game3': value_list[306], 
                    '2019-game4': value_list[307], 
                    '2019-game5': value_list[308], 
                    '2019-game6': value_list[309], 
                    '2019-game7': value_list[310], 
                    '2019-game8': value_list[311], 
                    '2019-game9': value_list[312],
                    '2019-game10': value_list[313],
                    '2019-game11': value_list[314],
                    '2019-game12': value_list[315],
                    '2019-game13': value_list[316],
                    '2019-game14': value_list[317],
                    '2019-game15': value_list[318],
                    '2019-game16': value_list[319]
                     }, ignore_index=True)
    
    return df

In [9]:
# The main code for iterating through the kicker list, calculating the points and adding the rows
# to the final_df dataframe.
column_names = ['player',
                'first',
                'last',
                'name',
                'position1', 
                '2000-game1',
                '2000-game2',
                '2000-game3', 
                '2000-game4', 
                '2000-game5', 
                '2000-game6', 
                '2000-game7', 
                '2000-game8', 
                '2000-game9',
                '2000-game10',
                '2000-game11',
                '2000-game12',
                '2000-game13',
                '2000-game14',
                '2000-game15',
                '2000-game16',
                '2001-game1', 
                '2001-game2',
                '2001-game3', 
                '2001-game4', 
                '2001-game5', 
                '2001-game6', 
                '2001-game7', 
                '2001-game8', 
                '2001-game9',
                '2001-game10',
                '2001-game11',
                '2001-game12',
                '2001-game13',
                '2001-game14',
                '2001-game15',
                '2001-game16',
                '2002-game1',
                '2002-game2',
                '2002-game3', 
                '2002-game4', 
                '2002-game5', 
                '2002-game6', 
                '2002-game7', 
                '2002-game8', 
                '2002-game9',
                '2002-game10',
                '2002-game11',
                '2002-game12',
                '2002-game13',
                '2002-game14',
                '2002-game15',
                '2002-game16',
                '2003-game1', 
                '2003-game2',
                '2003-game3', 
                '2003-game4', 
                '2003-game5', 
                '2003-game6', 
                '2003-game7', 
                '2003-game8', 
                '2003-game9',
                '2003-game10',
                '2003-game11',
                '2003-game12',
                '2003-game13',
                '2003-game14',
                '2003-game15',
                '2003-game16',
                '2004-game1',
                '2004-game2',
                '2004-game3', 
                '2004-game4', 
                '2004-game5', 
                '2004-game6', 
                '2004-game7', 
                '2004-game8', 
                '2004-game9',
                '2004-game10',
                '2004-game11',
                '2004-game12',
                '2004-game13',
                '2004-game14',
                '2004-game15',
                '2004-game16',
                '2005-game1', 
                '2005-game2',
                '2005-game3', 
                '2005-game4', 
                '2005-game5', 
                '2005-game6', 
                '2005-game7', 
                '2005-game8', 
                '2005-game9',
                '2005-game10',
                '2005-game11',
                '2005-game12',
                '2005-game13',
                '2005-game14',
                '2005-game15',
                '2005-game16',
                '2006-game1',
                '2006-game2',
                '2006-game3', 
                '2006-game4', 
                '2006-game5', 
                '2006-game6', 
                '2006-game7', 
                '2006-game8', 
                '2006-game9',
                '2006-game10',
                '2006-game11',
                '2006-game12',
                '2006-game13',
                '2006-game14',
                '2006-game15',
                '2006-game16',
                '2007-game1', 
                '2007-game2',
                '2007-game3', 
                '2007-game4', 
                '2007-game5', 
                '2007-game6', 
                '2007-game7', 
                '2007-game8', 
                '2007-game9',
                '2007-game10',
                '2007-game11',
                '2007-game12',
                '2007-game13',
                '2007-game14',
                '2007-game15',
                '2007-game16',
                '2008-game1',
                '2008-game2',
                '2008-game3', 
                '2008-game4', 
                '2008-game5', 
                '2008-game6', 
                '2008-game7', 
                '2008-game8', 
                '2008-game9',
                '2008-game10',
                '2008-game11',
                '2008-game12',
                '2008-game13',
                '2008-game14',
                '2008-game15',
                '2008-game16',
                '2009-game1', 
                '2009-game2',
                '2009-game3', 
                '2009-game4', 
                '2009-game5', 
                '2009-game6', 
                '2009-game7', 
                '2009-game8', 
                '2009-game9',
                '2009-game10',
                '2009-game11',
                '2009-game12',
                '2009-game13',
                '2009-game14',
                '2009-game15',
                '2009-game16',
                '2010-game1',
                '2010-game2',
                '2010-game3', 
                '2010-game4', 
                '2010-game5', 
                '2010-game6', 
                '2010-game7', 
                '2010-game8', 
                '2010-game9',
                '2010-game10',
                '2010-game11',
                '2010-game12',
                '2010-game13',
                '2010-game14',
                '2010-game15',
                '2010-game16',
                '2011-game1', 
                '2011-game2',
                '2011-game3', 
                '2011-game4', 
                '2011-game5', 
                '2011-game6', 
                '2011-game7', 
                '2011-game8', 
                '2011-game9',
                '2011-game10',
                '2011-game11',
                '2011-game12',
                '2011-game13',
                '2011-game14',
                '2011-game15',
                '2011-game16',
                '2012-game1',
                '2012-game2',
                '2012-game3', 
                '2012-game4', 
                '2012-game5', 
                '2012-game6', 
                '2012-game7', 
                '2012-game8', 
                '2012-game9',
                '2012-game10',
                '2012-game11',
                '2012-game12',
                '2012-game13',
                '2012-game14',
                '2012-game15',
                '2012-game16',
                '2013-game1', 
                '2013-game2',
                '2013-game3', 
                '2013-game4', 
                '2013-game5', 
                '2013-game6', 
                '2013-game7', 
                '2013-game8', 
                '2013-game9',
                '2013-game10',
                '2013-game11',
                '2013-game12',
                '2013-game13',
                '2013-game14',
                '2013-game15',
                '2013-game16',
                '2014-game1',
                '2014-game2',
                '2014-game3', 
                '2014-game4', 
                '2014-game5', 
                '2014-game6', 
                '2014-game7', 
                '2014-game8', 
                '2014-game9',
                '2014-game10',
                '2014-game11',
                '2014-game12',
                '2014-game13',
                '2014-game14',
                '2014-game15',
                '2014-game16',
                '2015-game1', 
                '2015-game2',
                '2015-game3', 
                '2015-game4', 
                '2015-game5', 
                '2015-game6', 
                '2015-game7', 
                '2015-game8', 
                '2015-game9',
                '2015-game10',
                '2015-game11',
                '2015-game12',
                '2015-game13',
                '2015-game14',
                '2015-game15',
                '2015-game16',
                '2016-game1',
                '2016-game2',
                '2016-game3', 
                '2016-game4', 
                '2016-game5', 
                '2016-game6', 
                '2016-game7', 
                '2016-game8', 
                '2016-game9',
                '2016-game10',
                '2016-game11',
                '2016-game12',
                '2016-game13',
                '2016-game14',
                '2016-game15',
                '2016-game16',
                '2017-game1', 
                '2017-game2',
                '2017-game3', 
                '2017-game4', 
                '2017-game5', 
                '2017-game6', 
                '2017-game7', 
                '2017-game8', 
                '2017-game9',
                '2017-game10',
                '2017-game11',
                '2017-game12',
                '2017-game13',
                '2017-game14',
                '2017-game15',
                '2017-game16',
                '2018-game1',
                '2018-game2',
                '2018-game3', 
                '2018-game4', 
                '2018-game5', 
                '2018-game6', 
                '2018-game7', 
                '2018-game8', 
                '2018-game9',
                '2018-game10',
                '2018-game11',
                '2018-game12',
                '2018-game13',
                '2018-game14',
                '2018-game15',
                '2018-game16',
                '2019-game1',
                '2019-game2',
                '2019-game3', 
                '2019-game4', 
                '2019-game5', 
                '2019-game6', 
                '2019-game7', 
                '2019-game8', 
                '2019-game9',
                '2019-game10',
                '2019-game11',
                '2019-game12',
                '2019-game13',
                '2019-game14',
                '2019-game15',
                '2019-game16'
                 ]

oplayers_df = player_df.loc[(player_df['position1'] == 'QB') |
                            (player_df['position1'] == 'RB') |
                            (player_df['position1'] == 'WR') |
                            (player_df['position1'] == 'TE')]
player_list = oplayers_df['player'].tolist()

final_df = pd.DataFrame(columns = column_names)

for player in player_list:

    first = player_df['first'].loc[(player_df['player']==player)].iloc[0]
    last = player_df['last'].loc[(player_df['player']==player)].iloc[0]
    name = player_df['name'].loc[(player_df['player']==player)].iloc[0]
    position1 = player_df['position1'].loc[(player_df['player']==player)].iloc[0]

    print(player)

    # Determine the current team of the player
    for s in range(1, 17):
        if offense_df['team'].loc[(offense_df['player']==player) & (offense_df['seas']==s)].empty:
            x = 0
        else:
            team = offense_df['team'].loc[(offense_df['player']==player) & (offense_df['seas']==s)].iloc[0]
            break
    new_team = team

    player_scores = []

    for year in range(2000,2020):
        week = 0
        for game in range(1,17):
            week = week + 1

            # Check the current team of the player
            if (offense_df['team'].loc[(offense_df['player']==player) & (offense_df['year']==year) & (offense_df['week']==week)].empty):
                x = 0
            else:
                new_team = offense_df['team'].loc[(offense_df['player']==player) & (offense_df['year']==year) & (offense_df['week']==week)].iloc[0]
            
            if team != new_team:
                team = new_team

            # Handle the bye week
            if ((game_df['h'].loc[(game_df['seas']==year) & (game_df['wk']==week) & ((game_df['v']==team) | (game_df['h']==team))].empty) &
                (game_df['v'].loc[(game_df['seas']==year) & (game_df['wk']==week) & ((game_df['v']==team) | (game_df['h']==team))].empty)):
                week = week + 1
                score = pull_stats(offense_df, player_df, td_df, 'fp', player, year, week)
            else:
                score = pull_stats(offense_df, player_df, td_df, 'fp', player, year, week)

            player_scores.append(score)

    final_df = add_row(final_df, player, first, last, name, position1, player_scores)

AD-0100
AM-1150
AR-1300
AT-0160
BA-0375
BB-2425
BH-1900
BH-2300
BM-0650
BR-1100
CD-0300
CK-0250
CM-1500
CN-0500
CW-1425
DB-2750
DB-3800
DC-0725
DH-1912
DH-3450
DJ-2275
DL-1787
DP-2037
DW-1725
EM-0200
GG-0310
GM-1350
JA-1087
JB-5725
JD-2850
JF-1900
JG-0650
JG-1850
JM-2900
JR-3250
JS-6787
JW-5650
KA-0737
KC-2350
KM-3500
LF-0025
LJ-0150
MB-0450
MG-0850
MM-0912
MM-4700
MR-2350
MR-2500
MS-0200
MS-4100
MT-1800
NF-0250
NM-0687
PM-0025
PR-0300
RF-0450
RF-0500
RG-1850
RG-1885
RT-0150
RW-3850
SD-0175
SM-0425
TB-2093
TB-2300
TB-2650
TH-1625
TM-2362
TS-1450
TT-0500
WG-0750
AA-0025
AA-0557
AB-1125
AE-0550
AI-0050
AJ-0375
AJ-1143
AK-0050
AM-0825
AM-2850
AP-0700
AS-0900
BB-2250
BH-1225
BP-1200
BS-0825
BS-1450
BS-2950
CA-0750
CC-0750
CC-2350
CE-0175
CG-1015
CH-0525
CH-5000
CM-1225
CP-2550
CT-1220
DB-2950
DC-3550
DD-0681
DF-1375
DF-1875
DG-3200
DH-1362
DH-2325
DH-2450
DH-3137
DJ-0887
DJ-1325
DJ-1850
DL-1700
DM-2787
DO-0025
DO-0600
DS-2177
DS-3300
DT-1231
DT-1992
DV-0550
DW-1350
DW-1550
DW-1750
DW-3125


In [10]:
final_df.head(50)

,player,first,last,name,position1,2000-game1,2000-game2,2000-game3,2000-game4,2000-game5,2000-game6,2000-game7,2000-game8,2000-game9,2000-game10,2000-game11,2000-game12,2000-game13,2000-game14,2000-game15,2000-game16,2001-game1,2001-game2,2001-game3,2001-game4,2001-game5,2001-game6,2001-game7,2001-game8,2001-game9,2001-game10,2001-game11,2001-game12,2001-game13,2001-game14,2001-game15,2001-game16,2002-game1,2002-game2,2002-game3,...,2017-game9,2017-game10,2017-game11,2017-game12,2017-game13,2017-game14,2017-game15,2017-game16,2018-game1,2018-game2,2018-game3,2018-game4,2018-game5,2018-game6,2018-game7,2018-game8,2018-game9,2018-game10,2018-game11,2018-game12,2018-game13,2018-game14,2018-game15,2018-game16,2019-game1,2019-game2,2019-game3,2019-game4,2019-game5,2019-game6,2019-game7,2019-game8,2019-game9,2019-game10,2019-game11,2019-game12,2019-game13,2019-game14,2019-game15,2019-game16
0,AD-0100,Andy,Dalton,Andy Dalton,QB,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,15.60,18.56,19.16,19.26,7.64,0.52,11.42,21.68,16.52,26.60,14.08,24.78,12.92,17.16,7.92,20.20,6.92,19.34,8.10,0,0,0,0,0,21.72,19.64,16.06,3.64,19.98,13.60,18.34,17.16,0,0,0,13.32,9.58,2.14,33.84,16.60
1,AM-1150,A.J.,McCarron,A.J. McCarron,QB,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.00,0.00,0.00,0.00,1.88,0.76,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.32,0.00,0.00,-0.2,0,0,0,0,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0,0,0,0.00,0.00,0.00,0.00,17.90
2,AR-1300,Aaron,Rodgers,Aaron Rodgers,QB,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.00,0.00,0.00,0.00,0.00,21.90,0.00,0.00,25.94,16.04,20.90,15.02,28.68,28.40,16.94,19.16,18.06,22.58,11.92,15.32,20.24,13.46,43.88,1.04,12.92,14.36,14.30,25.48,9.42,18.32,44.76,28.10,12.94,10.02,9.46,28.12,11.40,14.42,9.34,19.02
3,AT-0160,Alex,Tanney,Alex Tanney,QB,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0,0,0,0,0,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0,0,0,0.00,0.00,0.04,0.00,0.00
4,BA-0375,Brandon,Allen,Brandon Allen,QB,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0,0,0,0,0,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,18.02,13.4,2.08,0.00,0.00,0.00,0.00,0.00
5,BB-2425,Blake,Bortles,Blake Bortles,QB,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,17.32,8.56,22.60,25.06,19.52,25.04,22.38,3.22,13.24,33.54,8.90,25.32,20.60,10.16,1.44,19.74,22.60,3.86,8.98,0,0,0,4.06,3.78,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0,0.1,0,0.00,-0.88,0.00,0.00,0.00
6,BH-1900,Brian,Hoyer,Brian Hoyer,QB,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.00,1.48,0.00,0.00,-0.20,0.00,0.00,-0.10,0.00,0.00,0.00,-0.30,0.00,0.00,0.00,-0.20,0.00,0.48,0.00,0,0,0,-0.2,-0.3,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,15.32,6.16,-0.2,0.00,0.00,0.00,0.00,-0.20
7,BH-2300,Brett,Hundley,Brett Hundley,QB,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,14.08,3.46,24.40,7.96,25.70,0.00,6.00,9.38,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0,0,0,0,0,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0,0,0,0.00,0.00,0.00,5.46,0.60
8,BM-0650,Baker,Mayfield,Baker Mayfield,QB,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Na

In [0]:
# Save the results to .csv file
final_df.to_csv('/content/actuals_offense.csv', index=False)